In [1]:
# search.py
#This is Search v0 — the baseline retrieval system.

from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

In [2]:
# ---------------------------
# Config
# ---------------------------
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "google-io-transcripts"

EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
TOP_K = 5

In [3]:
# ---------------------------
# Init clients
# ---------------------------
q_client = QdrantClient(url=QDRANT_URL)

embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

In [33]:
# ---------------------------
# Vector search function
# ---------------------------
def vector_search(query: str, top_k: int = 5):
    query_vector = embedding_model.encode(
        query,
        normalize_embeddings=True
    ).tolist()

    results = q_client.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        limit=top_k,
        with_payload=True,      # 👈 IMPORTANT: this explicitly requests payloads 

        #You must explicitly request payloads if you want RAG.
        with_vectors=False
    )

    return results.points

In [34]:
# ---------------------------
# print helper
# ---------------------------
def print_results(results):
    for i, hit in enumerate(results, start=1):

        # Case 1: ScoredPoint
        if hasattr(hit, "payload"):
            payload = hit.payload
            score = hit.score

        # Case 2: tuple (point, score)
        elif isinstance(hit, tuple):
            point, score = hit
            payload = point.payload if hasattr(point, "payload") else None

        else:
            raise ValueError(f"Unexpected result type: {type(hit)}")

        print(f"\nResult {i}")
        print("-" * 40)
        print(f"Score: {score:.4f}")
        print(payload["text"][:500])


In [ ]:
query="What new AI models were announced at Google I/O 2025?"
query2="What is the most recent AI news?"

In [36]:
results = vector_search(query)

In [37]:
# ---------------------------
# CLI entry point
# ---------------------------
if __name__ == "__main__":
    query = input("\nEnter your query: ")
    results = vector_search(query)
    print_results(results)


Enter your query:  What new AI models were announced at Google I/O 2025?



Result 1
----------------------------------------
Score: 0.6365
and it runs it on a fleet of
real devices of different sizes just to verify that
it works really well. So if you're interested
in mobile development, check out Google AI Edge. Google AI Edge,
it's really cool. And with that, I'll
hand it over to Joana to talk about what's next. JOANA CARRASQUEIRA: Awesome. Thank you, Josh. And you've heard it in the
keynotes in the previous session with Demis and Sergey,
we're pushing the boundaries of what's possible to build with
AI here at Google and Google D

Result 2
----------------------------------------
Score: 0.6289
[MUSIC PLAYING] JOANA CARRASQUEIRA:
Hello, everyone. My name is Joana Carrasqueira,
and I lead Developer Relations at Google DeepMind. JOSH GORDON: Hi, everyone. I'm Josh. JOANA CARRASQUEIRA:
And we're very excited to welcome you to
our session, Google's AI Stack for Developers. We'll start by giving you a
quick overview of Google's AI stack. Who's at I/O for the fir

In [38]:
# debugging check
print(type(results))
print(type(results[0]))
print(results[0])

<class 'list'>
<class 'qdrant_client.http.models.models.ScoredPoint'>
id=17 version=1 score=0.6365168 payload={'doc_id': '4TE-KFXvhAk__chunk_017', 'video_id': '4TE-KFXvhAk', 'title': "Google I/O 2025 - What's New in AI", 'timestamp_start': 1920.12, 'timestamp_end': 2074.09, 'text': "and it runs it on a fleet of\nreal devices of different sizes just to verify that\nit works really well. So if you're interested\nin mobile development, check out Google AI Edge. Google AI Edge,\nit's really cool. And with that, I'll\nhand it over to Joana to talk about what's next. JOANA CARRASQUEIRA: Awesome. Thank you, Josh. And you've heard it in the\nkeynotes in the previous session with Demis and Sergey,\nwe're pushing the boundaries of what's possible to build with\nAI here at Google and Google DeepMind. And we're really excited to\nbring all this innovation and put it in the\nhands of developers, in the hands of the community. And it's never been a better\ntime to build and co-create together. So we